In [1]:
import pandas as pd

In [2]:
f = {0:'data/noCM_cinc.csv', # there is no credit 
    1:'data/noCM_cinn.csv', # there is no credit
    2:'data/CM_cinc.csv',
    3:'data/CM_cinn.csv',
    4:'data/noCM_B_cinc.csv',
    5:'data/noCM_B_cinn.csv'
    }

In [4]:
df = pd.read_csv('data/noCM_cinc.csv')

In [14]:
df.columns

Index(['PO#', 'Amount Paid', 'Payment Reference', 'Payment Date',
       'Payment Amount', 'Payment Type', 'Discount Percent Limit'],
      dtype='object')

In [15]:
# df.astype({'PO#':'string', 'Amount Paid':'int64'}).dtypes
df.head(2)

,PO#,Amount Paid,Payment Reference,Payment Date,Payment Amount,Payment Type,Discount Percent Limit
0,PO000091048593,1116.4,940195.0,27-01-2022,7373.19,ACH,2.0
1,PO000091049406,259.9,NaN,NaN,NaN,NaN,NaN


In [3]:
def withinTolerance(da:float, dl:float=2 ):
    '''
    dl: discount limit ; da: discount actual
    # Alex: On DL50,  Lowerbound::48, Upper: 52
    
    '''
    tolerance= 0.04
#     print(dl-dl*tolerance)
#     print(dl+dl*tolerance)
    return (dl-dl*tolerance <= da <= dl+dl*tolerance)#


In [20]:
CRPATH = f[2]
NSPATH = f[3]

crcsv = pd.read_csv(CRPATH).convert_dtypes()
cr = crcsv[['PO#', 'Amount Paid']]

crin = cr[cr['Amount Paid']>0] # CR records with Invoice
crcm = cr[cr['Amount Paid']<0]  # CR records with Credit

nscsv = pd.read_csv(NSPATH).convert_dtypes()
ns = nscsv.copy()
ns['po_clean'] = ns['PO/Check Number'].str.extract(r'(PO.{12})')

nsin = ns.query("Type=='Invoice'") # NS records with 
nscm = ns.query("Type=='Credit Memo'") # NS records with Credit Memo



CONST_payment_ref = crcsv.loc[0].get('Payment Reference')
CONST_payment_dt = crcsv.loc[0].get('Payment Date')
CONST_payment_amt = float(crcsv.loc[0].get('Payment Amount'))
CONST_payment_typ = crcsv.loc[0].get('Payment Type')
CONST_payment_disc_thresh = crcsv.loc[0].get('Discount Percent Limit')

CONST_count_of_credits  = cr[cr['Amount Paid']<0]['PO#'].count()
IS_credit_sheet_needed = CONST_count_of_credits > 0



In [21]:
errMessages = {'C0':'ALERT: Credit memo NOT FOUND in CUSTOMER REMITTANCE data \nProcessing Payment Sheet...',
              'C1':'ALERT: Credit memo FOUND, Processing credit sheet...',
              'N0':'ERROR: No credit memo reference found in NETSUITE data, \nPlease contact Admin \nExiting program...',
              'M0':'ERROR: Credit Memo amount Mismatch \nPlease contact Admin \nExiting program...',
               'N1':'ALERT: Credit memo exists in NETSUITE data, Processing credit sheet...',
               'D0':'ERROR: Discount Tolerence Error \nPlease contact Admin \nExiting program...',
               'P0':'ERROR: Sum of Invoice Application does not equal Payment amount, Exiting program...',
               'P1':'ERROR: Sum of Invoice Application + Discount does not equal sum Amount Remaning, Exiting program...'
              
             }

In [22]:
CONST_payment_amt

6135.62

In [23]:
# Working for Payment sheet with MODE: No credit

In [24]:
crnsINp = (pd.merge(crin, nsin, how='left', left_on='PO#', right_on='po_clean', suffixes=('_cr', '_ns'))
 .get(['Date', 'Customer Internal ID', 'Internal ID', 'PO#', 'Amount Remaining', 'Amount Paid', 'Document Number'])
 .assign(**{'Discount Amount':lambda x: abs(x['Amount Paid'] - x['Amount Remaining']),
            'Discount Percent':lambda x:  (x['Discount Amount'] / x['Amount Remaining']).round(2).mul(100), 
            'WithinTolerence':lambda x: x['Discount Percent'].apply(withinTolerance, args=(2,)).map({True:'Yes', False:'No'}), # within tolerance of +- 20%  of 2 %
            'Invoice Application': lambda x: x['Amount Paid'],
            'Payment #': CONST_payment_ref,
            'Payment Amount': CONST_payment_amt,
            'External ID': f"PAACH{CONST_payment_ref}",
            'Memo':f"ACH {CONST_payment_ref}"
           }).sort_values(by='Amount Paid', ascending=False).reset_index(drop=True)
    
)


crnsCMp = (pd.merge(crcm, nscm, how='left', left_on='PO#', right_on='po_clean', suffixes=('_cr', '_ns'))
 .get(['Date', 'Customer Internal ID', 'Internal ID', 'PO#', 'Amount Remaining', 'Amount Paid', 'Amount', 'Document Number'])
 .assign(**{  'External ID' : f'CAACH{CONST_payment_ref}',
              'absAmount' : lambda x: abs(x['Amount Paid']),
#               'Discount Amount':lambda x: abs(x['Amount Paid'] - x['Amount Remaining']),
#             'Discount Percent':lambda x:  (x['Discount Amount'] / x['Amount Remaining']).round(2).mul(100), 
#             'WithinTolerence':lambda x: x['Discount Percent'].apply(withinTolerance, args=(2,)).map({True:'Yes', False:'No'}) # within tolerance of +- 20%  of 2 %
           })
           .sort_values(by='Amount Paid', ascending=True).reset_index(drop=True)
    
)


In [25]:
crnsCMp

,Date,Customer Internal ID,Internal ID,PO#,Amount Remaining,Amount Paid,Amount,Document Number,External ID,absAmount
0,10-12-2021,4871,2324010,PO000090262133,708.27,-708.27,-708.27,CM40559199,CAACH940195,708.27
1,21-02-2022,4871,3263953,PO000091310435,529.3,-529.3,-529.3,CM40559811,CAACH940195,529.3


In [26]:
crnsINp

,Date,Customer Internal ID,Internal ID,PO#,Amount Remaining,Amount Paid,Document Number,Discount Amount,Discount Percent,WithinTolerence,Invoice Application,Payment #,Payment Amount,External ID,Memo
0,21-01-2022,4871,2931735,PO000091048593,1139.18,1116.4,I260702483,22.78,2.0,Yes,1116.4,940195,6135.62,PAACH940195,ACH 940195
1,24-01-2022,4871,2940637,PO000091058279,877.61,860.06,I260702539,17.55,2.0,Yes,860.06,940195,6135.62,PAACH940195,ACH 940195
2,24-01-2022,4871,2940734,PO000091071246,874.35,856.86,I260702538,17.49,2.0,Yes,856.86,940195,6135.62,PAACH940195,ACH 940195
3,24-01-2022,4871,2940670,PO000091077370,708.27,694.1,I260702587,14.17,2.0,Yes,694.1,940195,6135.62,PAACH940195,ACH 940195
4,24-01-2022,4871,2940651,PO000091051395,653.18,640.12,I260702558,13.06,2.0,Yes,640.12,940195,6135.62,PAACH940195,ACH 940195
5,24-01-2022,4871,2940652,PO000091066487,485.23,475.53,I260702560,9.7,2.0,Yes,475.53,940195,6135.62,PAACH940195,ACH 940195
6,24-01-2022,4871,2940641,PO000091058608,398.2,390.24,I260702545,7.96,2.0,Yes,390.24,940195,6135.62,PAACH940195,ACH 940195
7,24-01-2022,4871,2940749,PO000091066102,398.2,390.24,I260702565,7.96,2.0,Yes,390.24,940195,6135.62,PAACH940195,ACH 940195
8,24-01-2022,4871,2940640,PO000091068056,398.2,390.24,I260702543,7.96,2.0,Yes,390.24,940195,6135.62,PAACH940195,ACH 940195
9,24-01-2022,4871,2940745,PO000091049406,265.2,259.9,I260702559,5.3,2.0,Yes,259.9,940195,6135.62,PAACH940195,ACH 940195


In [27]:
# Credit Application when CM < any one Bill


# Credit Application when CM > any one Bill


In [28]:
nscmP= (pd.merge(crcm, nscm, how='left', left_on='PO#', right_on='po_clean', suffixes=('_cr', '_ns'))
 .get(['Date', 'Customer Internal ID', 'Internal ID', 'PO#', 'Amount Remaining', 'Amount Paid', 'Document Number'])
#  .assign(**{'Discount Amount':lambda x: abs(x['Amount Paid'] - x['Amount Remaining']),
#             'Discount Percent':lambda x:  (x['Discount Amount'] / x['Amount Remaining']).round(2).mul(100), 
#             'WithinTolerence':lambda x: x['Discount Percent'].apply(withinTolerance, args=(2,)).map({True:'Yes', False:'No'}) # within tolerance of +- 20%  of 2 %
#            })
    
)

In [29]:
def apply_credit(crnsINp, crnsCMp):
    '''
    If all amounts in credit memo sheet are less than maximum of invoice amount then
    apply credit from top to bottom 
    '''
    crnsINcomp = crnsINp.copy(deep=True)
    CONST_MAX_INV = crnsINp.sort_values(by='Amount Paid', ascending=False)['Amount Paid'].max()

    if all(abs(y) < CONST_MAX_INV for y in crnsCMp['Amount'].to_list() ):
        for index, row in crnsCMp[['Internal ID', 'Amount', 'Document Number']].iterrows():
            # adding credit to invoice (meaning deducting the amount since the amt is a negative number)
            crnsINcomp.at[index, 'Invoice Application'] = crnsINcomp.at[index, 'Invoice Application'] + row['Amount'] 
            crnsINcomp.at[index, 'Credit Memo'] = row['Document Number']
            crnsINcomp.at[index, 'Credit Internal'] = row['Internal ID']
#             crnsINcomp.at[index, 'Invoice Application'] = abs(row['Amount'])
            crnsINcomp.at[index, 'Credit Amount'] = abs(row['Amount'])
        # Out of for loop
        return crnsINcomp 
    
    else:
        return pd.DataFrame() # "Credit is greater than INV maximum"



In [30]:
ccc = apply_credit(crnsINp, crnsCMp)

In [31]:
ccc['Invoice Application'].sum().round(2)

6135.62

In [38]:
def processing_discount_again(ccc, CONST_payment_ref, CONST_payment_amt):
    pm = (ccc
    [['Date', 'Customer Internal ID', 'Internal ID', 'PO#', 'Amount Remaining', 'Amount Paid', 'Invoice Application', 'Payment #', 'Payment Amount', 'External ID', 'Memo']]
    .assign(**{'Discount Amount':lambda x: abs(x['Amount Paid'] - x['Amount Remaining']),
                'Discount Percent':lambda x:  (x['Discount Amount'] / x['Amount Remaining']).round(2).mul(100), 
                'WithinTolerence':lambda x: x['Discount Percent'].apply(withinTolerance, args=(2,)).map({True:'Yes', False:'No'}), # within tolerance of +- 20%  of 2 %

            }).sort_values(by='Amount Paid', ascending=False)
        
    )
    return pm
g = payment_sheet_processing_with_credit(ccc, CONST_payment_ref, CONST_payment_amt)#['Invoice Application'].sum().round()


In [ ]:
#                 'Payment #': CONST_payment_ref,
#                 'Payment Amount': CONST_payment_amt,
#                 'External ID': f"PAACH{CONST_payment_ref}",
#                 'Memo':f"ACH {CONST_payment_ref}"

In [40]:
g

,Date,Customer Internal ID,Internal ID,PO#,Amount Remaining,Amount Paid,Invoice Application,Payment #,Payment Amount,External ID,Memo,Discount Amount,Discount Percent,WithinTolerence
0,21-01-2022,4871,2931735,PO000091048593,1139.18,1116.4,408.13,940195,6135.62,PAACH940195,ACH 940195,22.78,2.0,Yes
1,24-01-2022,4871,2940637,PO000091058279,877.61,860.06,330.76,940195,6135.62,PAACH940195,ACH 940195,17.55,2.0,Yes
2,24-01-2022,4871,2940734,PO000091071246,874.35,856.86,856.86,940195,6135.62,PAACH940195,ACH 940195,17.49,2.0,Yes
3,24-01-2022,4871,2940670,PO000091077370,708.27,694.1,694.1,940195,6135.62,PAACH940195,ACH 940195,14.17,2.0,Yes
4,24-01-2022,4871,2940651,PO000091051395,653.18,640.12,640.12,940195,6135.62,PAACH940195,ACH 940195,13.06,2.0,Yes
5,24-01-2022,4871,2940652,PO000091066487,485.23,475.53,475.53,940195,6135.62,PAACH940195,ACH 940195,9.7,2.0,Yes
6,24-01-2022,4871,2940641,PO000091058608,398.2,390.24,390.24,940195,6135.62,PAACH940195,ACH 940195,7.96,2.0,Yes
7,24-01-2022,4871,2940749,PO000091066102,398.2,390.24,390.24,940195,6135.62,PAACH940195,ACH 940195,7.96,2.0,Yes
8,24-01-2022,4871,2940640,PO000091068056,398.2,390.24,390.24,940195,6135.62,PAACH940195,ACH 940195,7.96,2.0,Yes
9,24-01-2022,4871,2940745,PO000091049406,265.2,259.9,259.9,940195,6135.62,PAACH940195,ACH 940195,5.3,2.0,Yes


In [50]:
prep_cosmetic_payment_sheet(crnsINp)

,Date,Customer ID,Payment #,Invoice Internal,Payment Amount,Invoice Application,Discount,External ID,Memo
0,21-01-2022,4871,940195,2931735,6135.62,1116.4,22.78,PAACH940195,ACH 940195
1,24-01-2022,4871,940195,2940637,6135.62,860.06,17.55,PAACH940195,ACH 940195
2,24-01-2022,4871,940195,2940734,6135.62,856.86,17.49,PAACH940195,ACH 940195
3,24-01-2022,4871,940195,2940670,6135.62,694.1,14.17,PAACH940195,ACH 940195
4,24-01-2022,4871,940195,2940651,6135.62,640.12,13.06,PAACH940195,ACH 940195
5,24-01-2022,4871,940195,2940652,6135.62,475.53,9.7,PAACH940195,ACH 940195
6,24-01-2022,4871,940195,2940641,6135.62,390.24,7.96,PAACH940195,ACH 940195
7,24-01-2022,4871,940195,2940749,6135.62,390.24,7.96,PAACH940195,ACH 940195
8,24-01-2022,4871,940195,2940640,6135.62,390.24,7.96,PAACH940195,ACH 940195
9,24-01-2022,4871,940195,2940745,6135.62,259.9,5.3,PAACH940195,ACH 940195


In [45]:
def prep_cosmetic_payment_sheet(df):
    df = df.rename(columns={'Customer Internal ID':'Customer ID',
                            'Internal ID': 'Invoice Internal', 
                            'Discount Amount': 'Discount',
                      })
    
    df = df[['Date', 'Customer ID', 'Payment #', 'Invoice Internal', 'Payment Amount', 'Invoice Application', 'Discount', 'External ID', 'Memo']]
#     df['Date'] = df.Date.dt.strftime('%m/%d/%Y')
    return df
a = prep_cosmetic_payment_sheet(g)
a

,Date,Customer ID,Payment #,Invoice Internal,Payment Amount,Invoice Application,Discount,External ID,Memo
0,21-01-2022,4871,940195,2931735,6135.62,408.13,22.78,PAACH940195,ACH 940195
1,24-01-2022,4871,940195,2940637,6135.62,330.76,17.55,PAACH940195,ACH 940195
2,24-01-2022,4871,940195,2940734,6135.62,856.86,17.49,PAACH940195,ACH 940195
3,24-01-2022,4871,940195,2940670,6135.62,694.1,14.17,PAACH940195,ACH 940195
4,24-01-2022,4871,940195,2940651,6135.62,640.12,13.06,PAACH940195,ACH 940195
5,24-01-2022,4871,940195,2940652,6135.62,475.53,9.7,PAACH940195,ACH 940195
6,24-01-2022,4871,940195,2940641,6135.62,390.24,7.96,PAACH940195,ACH 940195
7,24-01-2022,4871,940195,2940749,6135.62,390.24,7.96,PAACH940195,ACH 940195
8,24-01-2022,4871,940195,2940640,6135.62,390.24,7.96,PAACH940195,ACH 940195
9,24-01-2022,4871,940195,2940745,6135.62,259.9,5.3,PAACH940195,ACH 940195


In [44]:
ccc['Invoice Application'].sum() + ccc['Discount Amount'].sum()

6286.049999999998

In [36]:
g['Invoice Application'].sum() + g['Discount Amount'].sum()

6286.049999999998

In [49]:
g['Invoice Application'].sum()

6135.619999999998

In [48]:
CONST_payment_amt

6135.62

In [37]:
CONST_payment_amt

6135.62

In [41]:
crnsINp['Amount Paid'].sum()

7373.189999999998

In [42]:
crnsINp

,Date,Customer Internal ID,Internal ID,PO#,Amount Remaining,Amount Paid,Document Number,Discount Amount,Discount Percent,WithinTolerence,Invoice Application
0,21-01-2022,4871,2931735,PO000091048593,1139.18,1116.4,I260702483,22.78,2.0,Yes,1116.4
1,24-01-2022,4871,2940637,PO000091058279,877.61,860.06,I260702539,17.55,2.0,Yes,860.06
2,24-01-2022,4871,2940734,PO000091071246,874.35,856.86,I260702538,17.49,2.0,Yes,856.86
3,24-01-2022,4871,2940670,PO000091077370,708.27,694.1,I260702587,14.17,2.0,Yes,694.1
4,24-01-2022,4871,2940651,PO000091051395,653.18,640.12,I260702558,13.06,2.0,Yes,640.12
5,24-01-2022,4871,2940652,PO000091066487,485.23,475.53,I260702560,9.7,2.0,Yes,475.53
6,24-01-2022,4871,2940641,PO000091058608,398.2,390.24,I260702545,7.96,2.0,Yes,390.24
7,24-01-2022,4871,2940749,PO000091066102,398.2,390.24,I260702565,7.96,2.0,Yes,390.24
8,24-01-2022,4871,2940640,PO000091068056,398.2,390.24,I260702543,7.96,2.0,Yes,390.24
9,24-01-2022,4871,2940745,PO000091049406,265.2,259.9,I260702559,5.3,2.0,Yes,259.9


In [43]:
ccc

,Date,Customer Internal ID,Internal ID,PO#,Amount Remaining,Amount Paid,Document Number,Discount Amount,Discount Percent,WithinTolerence,Invoice Application,Credit Memo,Credit Internal,Credit Amount
0,21-01-2022,4871,2931735,PO000091048593,1139.18,1116.4,I260702483,22.78,2.0,Yes,408.13,CM40559199,2324010.0,708.27
1,24-01-2022,4871,2940637,PO000091058279,877.61,860.06,I260702539,17.55,2.0,Yes,330.76,CM40559811,3263953.0,529.30
2,24-01-2022,4871,2940734,PO000091071246,874.35,856.86,I260702538,17.49,2.0,Yes,856.86,NaN,NaN,NaN
3,24-01-2022,4871,2940670,PO000091077370,708.27,694.1,I260702587,14.17,2.0,Yes,694.1,NaN,NaN,NaN
4,24-01-2022,4871,2940651,PO000091051395,653.18,640.12,I260702558,13.06,2.0,Yes,640.12,NaN,NaN,NaN
5,24-01-2022,4871,2940652,PO000091066487,485.23,475.53,I260702560,9.7,2.0,Yes,475.53,NaN,NaN,NaN
6,24-01-2022,4871,2940641,PO000091058608,398.2,390.24,I260702545,7.96,2.0,Yes,390.24,NaN,NaN,NaN
7,24-01-2022,4871,2940749,PO000091066102,398.2,390.24,I260702565,7.96,2.0,Yes,390.24,NaN,NaN,NaN
8,24-01-2022,4871,2940640,PO000091068056,398.2,390.24,I260702543,7.96,2.0,Yes,390.24,NaN,NaN,NaN
9,24-01-2022,4871,2940745,PO000091049406,265.2,259.9,I260702559,5.3,2.0,Yes,259.9,NaN,NaN,NaN


In [ ]:
ccc['Amount Paid'].sum()

In [19]:
def prep_cosmetic_credit_sheet(ccc, CONST_payment_ref):
    df = ccc.get(['Credit Memo', 'Credit Internal', 'Document Number', 'Internal ID', 'Invoice Application', 'Credit Amount']).assign(**{'External ID':f'CAACH{CONST_payment_ref}'}).dropna()
    df = df.rename(columns={'Document Number':'Invoice Number',
                            'Internal ID':'Applied Invoice'
                           }).round(3)
    df = df.astype('string')
    return df
prep_cosmetic_credit_sheet(ccc, CONST_payment_ref)

,Credit Memo,Credit Internal,Invoice Number,Applied Invoice,Invoice Application,Credit Amount,External ID
0,CM40559199,2324010.0,I260702483,2931735,408.13,708.27,CAACH940195
1,CM40559811,3263953.0,I260702539,2940637,330.76,529.3,CAACH940195


In [ ]:
crnsINp